In [18]:
instances = [
    "http://localhost:8001/api/store/",
    "http://localhost:8002/api/store/",
    # Добавьте больше инстансов, если нужно
]
def quorum_delete(key):
    """Удаление данных с кворумом."""
    responses = []
    for instance in instances:
        try:
            response = requests.delete(f"{instance}{key}/delete/")  # Передаем параметр базы данных
            responses.append(response)
        except requests.exceptions.RequestException as e:
            print(f"Ошибка при удалении с {instance}: {e}")

    # Проверка, что большинство удалений успешны
    successes = sum(1 for r in responses if r.status_code == 204)
    return successes > len(responses) // 2

def quorum_write(key, value):
    """Запись данных с кворумом."""
    responses = []
    for instance in instances:
        try:
            response = requests.post(instance, json={"key": key, "value": value})  # Передаем параметр базы данных
            responses.append(response)
        except requests.exceptions.RequestException as e:
            print(f"Ошибка при записи на {instance}: {e}")

    # Проверка, что большинство записей успешны
    successes = sum(1 for r in responses if r.status_code == 201)
    return successes > len(responses) // 2

def quorum_read(key):
    """Чтение данных с кворумом."""
    values = []
    for instance in instances:
        try:
            response = requests.get(f"{instance}{key}")  # Передаем параметр базы данных
            if response.status_code == 200:
                values.append(response.json().get('value'))
        except requests.exceptions.RequestException as e:
            print(f"Ошибка при чтении с {instance}: {e}")

In [20]:
import time

key = "example_key"
value = "example_value"

print(f"Удаление данных для ключа: {key}")
if quorum_delete(key):
    print("Данные успешно удалены с кворумом.")
else:
    print("Ошибка удаления данных с кворумом или данные не существовали.")

print(f"Запись данных: {key} = {value}")
if quorum_write(key, value):
    print("Данные успешно записаны с кворумом.")
else:
    print("Ошибка записи данных с кворумом.")

# Подождите некоторое время, чтобы убедиться, что данные успели записаться
time.sleep(2)

print(f"Чтение данных для ключа: {key}")
read_value = quorum_read(key)
if read_value is not None:
    print(f"Успешно прочитано: {key} = {read_value}")
else:
    print("Ошибка чтения данных.")

Удаление данных для ключа: example_key
Ошибка удаления данных с кворумом или данные не существовали.
Запись данных: example_key = example_value
Данные успешно записаны с кворумом.
Чтение данных для ключа: example_key
Ошибка чтения данных.
